In [44]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset
from sklearn.preprocessing import StandardScaler

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

In [45]:
df = pd.read_csv("LIAR2/train.csv")
display(df.head())
print(len(df["subject"].unique()))

,id,label,statement,date,subject,speaker,speaker_description,state_info,true_counts,mostly_true_counts,half_true_counts,mostly_false_counts,false_counts,pants_on_fire_counts,context,justification
0,13847,5,"90 percent of Americans ""support universal bac...","October 2, 2017",government regulation;polls and public opinion...,chris abele,"Chris Abele is Milwaukee County Executive, a p...",wisconsin,1,4,5,3,5,2,a tweet,"""Universal"" is the term for background checks ..."
1,13411,1,Last year was one of the deadliest years ever ...,"May 19, 2017",after the fact;congress;criminal justice;histo...,thom tillis,Thom Tillis is a Republican who serves as U.S....,north carolina,0,2,7,3,2,0,a press release supporting the Back The Blue A...,"Sen. Thom Tillis, a North Carolina Republican,..."
2,10882,0,"Bernie Sanders's plan is ""to raise your taxes ...","October 28, 2015",taxes,chris christie,"Chris Christie announced June 6, 2023 that he ...",national,21,20,27,11,17,8,"Boulder, Colo","Christie said that Sanders’s plan is ""to raise..."
3,20697,4,Voter ID is supported by an overwhelming major...,"December 8, 2021",voter id laws,lee zeldin,Lee Zeldin is a Republican representing New Yo...,new york,1,2,0,0,0,0,a Tweet,Zeldin claimed voter identification requiremen...
4,6095,2,"Says Barack Obama ""robbed Medicare (of) $716 b...","August 12, 2012",federal budget;history;medicare;retirement,mitt romney,Mitt Romney is a U.S. senator from Utah. He ra...,national,31,33,58,35,32,19,"an interview on ""60 Minutes""","Romney said, ""There's only one president that ..."


6049


In [46]:
"""lst = []
df["subject"] = df["subject"].fillna("NONE")
for object in df["subject"]:
    item = object.split(";")
    
    lst.extend(item)

unique_items = set(lst)
print(len(lst))
print(len(unique_items))"""

'lst = []\ndf["subject"] = df["subject"].fillna("NONE")\nfor object in df["subject"]:\n    item = object.split(";")\n    \n    lst.extend(item)\n\nunique_items = set(lst)\nprint(len(lst))\nprint(len(unique_items))'

In [47]:


def build_descriptive_text_vocab_subject_stateInfo(input_text):
    vocab = set()
    vocab.add("<UNK>")
    
    # First replace NaN values with "<NONE>"
    input_text = input_text.fillna("<NONE>")
    
    # Convert all entries to string
    input_text = input_text.astype(str)
    
    # Build vocabulary
    for text in input_text:
        for word in text.split(";"):
            word = word.strip()  # Remove extra spaces
            if word:
                vocab.add(word)
    
    # Return the vocabulary as a dictionary
    return {token: i for i, token in enumerate(vocab)}

def vectorize_descriptive_text_subject(input_text, vocab):
  vectorized_text = np.zeros(len(vocab))
  for word in input_text.split():
    # word = remove_punctuation(word)
    if word in vocab:
      vectorized_text[vocab[word]] += 1
    else:
      vectorized_text[vocab["<UNK>"]] += 1
  return vectorized_text

V_subject = build_descriptive_text_vocab_subject_stateInfo(df["subject"])
V_state = build_descriptive_text_vocab_subject_stateInfo(df["state_info"])

print(V_subject)

check = vectorize_descriptive_text_subject("bankruptcy", V_subject)
print(len(check))
print(check)



{'voting record': 0, 'bankruptcy': 1, 'party support': 2, 'facebook fact-checks': 3, 'autism': 4, 'oregon': 5, 'texas': 6, 'ohio': 7, 'infrastructure': 8, 'punditfact': 9, 'women': 10, 'natural disasters': 11, 'ad watch': 12, 'redistricting': 13, 'coronavirus': 14, "the 2018 california governor's race": 15, 'social security': 16, 'regulation': 17, 'state budget': 18, 'states': 19, 'iowa': 20, 'water': 21, 'taxes': 22, 'negative campaigning': 23, 'public health': 24, 'children': 25, 'congress': 26, 'privacy issues': 27, 'ebola': 28, 'federal budget': 29, 'technology': 30, 'environment': 31, 'recreation': 32, 'campaign finance': 33, 'economy': 34, 'florida': 35, 'food safety': 36, 'county budget': 37, 'elections': 38, 'homeland security': 39, 'retirement': 40, 'religion': 41, 'israel': 42, 'abortion': 43, 'north carolina': 44, 'lgbtq': 45, 'drugs': 46, 'urban': 47, 'sexuality': 48, 'public safety': 49, 'criminal justice': 50, 'small business': 51, 'fake news': 52, 'consumer safety': 53, 

In [ ]:
def build_descriptive_text_vocab(input_text):
  vocab = set()
  vocab.add("<UNK>")
  input_text = input_text.astype(str)
  input_text = input_text.fillna("<NONE>")
  for text in input_text:
    for word in text.split(";"):
      if word:
        vocab.add(word)
  return {token: i for i, token in enumerate(vocab)}

def vectorize_descriptive_text(input_text, vocab):
  vectorized_text = np.zeros(len(vocab))
  for word in input_text.split():
    word = remove_punctuation(word)
    if word in vocab:
      vectorized_text[vocab[word]] += 1
    else:
      vectorized_text[vocab["<UNK>"]] += 1
  return vectorized_text

train_data = pd.read_csv('LIAR2/train.csv')
V = build_descriptive_text_vocab(train_data['statement'])

test_statement = "percent chores chores stunt chores stunt chores ontario the"
test_vector = vectorize_descriptive_text(test_statement, V)
print(test_vector)
for i in test_vector:
  print(i)

{'Said, "The Seven Years\' War\xa0led to near bankruptcy for many countries': 0, 'Says a highway was closed in El Paso because of bullets flying across the border.': 1, 'Says in 2002, he "won more Hispanic votes than Anglo votes, 60 percent in the state.': 2, 'Says Keanu Reeves gave a speech that began, "I cannot be part of a world where men dress their wives as prostitutes by showing everything that should be cherished ...': 3, 'Says using E-Verify is "a fair amount of cost," about $20 per person that does not match.': 4, 'Tim Kaine hiked tuition as governor, but now claims to champion affordability.': 5, 'Under Donald Trump\'s tax plan, "the top 0.1 percent of taxpayers -- people earning multiple millions of dollars a year, on average -- \xa0would get more tax relief than the bottom 60 percent of taxpayers combined.': 6, "We're losing now over $500 billion a year in terms of imbalance with China.": 7, 'Bob Corker gave us the Iran Deal.': 8, 'Hillary stood up for universal health care

class CustomCSVDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        """
        Args:
            csv_file (str): Path to the CSV file containing the data.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.data = pd.read_csv(csv_file)  # Load the CSV file
        self.transform = transform

        # Assuming the last column is the label
        self.features = self.data.iloc[:, :-1].values  # All columns except the last one (features)
        self.labels = self.data.iloc[:, -1].values    # The last column (label)

        # Standardize features if necessary
        self.scaler = StandardScaler()
        self.features = self.scaler.fit_transform(self.features)  # Normalize features

    def __len__(self):
        """Returns the total number of samples."""
        return len(self.data)

    def __getitem__(self, idx):
        """
        Args:
            idx (int): Index of the sample to fetch.
        Returns:
            tuple: (features, label) where features are the input tensor and label is the output tensor.
        """
        features = torch.tensor(self.features[idx], dtype=torch.float32)
        label = torch.tensor(self.labels[idx], dtype=torch.float32)

        if self.transform:
            features = self.transform(features)

        return features, label
